# Typical Start

`nbdev` needs ot know where to export the module code fomr this notebook. You will typically just export to a module name the same as the notebook but you can rename and add extra directories to create a package structure if you wish

In [1]:
# hide
# default_exp hello_sciflow

The below code is an example of how to define a step. A step is a function we chain together to form a workflow. Typically they will have inputs and return values which can be used in subsequent steps but we start simple here.

## Breaking down the named step directive:

* `exportn_step:first` = export a named step called **first**

In [2]:
# exportn_step:first


def first():
    print("The first step")

Auto-reloading modules is very useful when using `nbdev` as changes to underlying modules are picked up without having to restart the kernel.

In [3]:
%load_ext autoreload
%autoreload 2

You need to export all code that is gonig to be used in flows - this means things like import statements as well as function definitions.

In [4]:
# export

# Params

* `sciflow` uses the papermill format for paramaeterising notebooks. 
* See here for how to specify papermill params: https://papermill.readthedocs.io/en/latest/usage-parameterize.html. 
* These parameters will be available to use or edit in your flows.

In [5]:
# export

traffic_percent = 1

In [6]:
# exportn_step:second


def second(traffic_percent):
    results = {}
    # Currently step functions need to return a dictionary. The keys: artifacts and metrics are reserved for experiment tracking otherwiose any other named keys will be available in future steps.
    results["traffic_multiplied"] = traffic_percent * 2
    return results

# See the results

## 1. Command Line Interface

# 1.1  Convert the notebooks to sciflow modules

In [7]:
# tst
!sciflow_build_lib

Converted 01_hello_nbdev.ipynb.
Converted 02_hello_sciflow.ipynb.
Converted complete_workflow.ipynb.
Converted experiment_tracking.ipynb.
Converted running_metaflows.ipynb.
Converted tidying_notebooks.ipynb.
Converted why_make.ipynb.


# 1.2 Tests that the notebooks work!

In [8]:
# tst
!nbdev_test_nbs --pause=1

testing /home/jovyan/git/sciflow/examples/01_hello_nbdev.ipynb
testing /home/jovyan/git/sciflow/examples/complete_workflow.ipynb
/opt/conda/envs/sciflow/lib/python3.7/site-packages/nbdev/test.py:97: RuntimeWarning: coroutine 'ZMQSocketChannel.get_msg' was never awaited
  return False,time.time()-start
testing /home/jovyan/git/sciflow/examples/02_hello_sciflow.ipynb
/opt/conda/envs/sciflow/lib/python3.7/site-packages/nbdev/test.py:97: RuntimeWarning: coroutine 'ZMQSocketChannel.get_msg' was never awaited
  return False,time.time()-start
[IPKernelApp] ERROR | KeyboardInterrupt caught in kernel.
/opt/conda/envs/sciflow/lib/python3.7/site-packages/nbdev/test.py:97: RuntimeWarning: coroutine 'ZMQSocketChannel.get_msg' was never awaited
  return False,time.time()-start
testing /home/jovyan/git/sciflow/examples/running_metaflows.ipynb
/opt/conda/envs/sciflow/lib/python3.7/site-packages/nbdev/test.py:97: RuntimeWarning: coroutine 'ZMQSocketChannel.get_msg' was never awaited
  return False,time

# 1.3 See the created `Flow`

In [11]:
# tst
!cat sciflow_examples/hello_sciflow.py

# AUTOGENERATED! DO NOT EDIT! File to edit: 02_hello_sciflow.ipynb (unless otherwise specified).

__all__ = ['first', 'traffic_percent', 'second']

# step:first


def first():
    print("The first step")

# Cell


# Cell

traffic_percent = 1

# step:second


def second(traffic_percent):
    results = {}
    # Currently step functions need to return a dictionary. The keys: artifacts and metrics are reserved for experiment tracking otherwiose any other named keys will be available in future steps.
    results["traffic_multiplied"] = traffic_percent * 2
    return results

# Run a `Flow`, this example uses `metaflow`

In [12]:
##tst
!source ~/.sciflow/env && python sciflow_examples/hello_sciflow.py --no-pylint run

# 2. Programmatically

TBC..